In [23]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import fiona



data_dir = '../data'

# Load the data
tree_dataset = gpd.read_file(data_dir + '/raw_data/geodata_stadt_Zuerich/trees/data/data.gpkg')
flats_duration = gpd.read_file(data_dir + '/derived_data/flats_duration.gpkg')

fiona.listlayers(data_dir + '/raw_data/geodata_stadt_Zuerich/vbz/data/data.gpkg')



['vbz_infrastruktur_ogd.strecke_vbz_kunstbauten_l',
 'vbz_infrastruktur_ogd.strecke_schweissstoss',
 'vbz_infrastruktur_ogd.strecke_weichensignal',
 'vbz_infrastruktur_ogd.strecke_esaschmieranlage',
 'vbz_infrastruktur_ogd.strecke_antennentopf',
 'vbz_infrastruktur_ogd.strecke_tr_weiche',
 'vbz_infrastruktur_ogd.strecke_gleiskreisanschlusskas',
 'vbz_infrastruktur_ogd.fahrleitungen_ausleger',
 'vbz_infrastruktur_ogd.strecke_weichenantriebskasten',
 'vbz_infrastruktur_ogd.fahrleitungen_feeder',
 'vbz_infrastruktur_ogd.strecke_ausgleichvorrichtung',
 'vbz_infrastruktur_ogd.fahrleitungen_fl_kreuzung',
 'vbz_infrastruktur_ogd.strecke_radlenker',
 'vbz_infrastruktur_ogd.strecke_pfahlgruendung',
 'vbz_infrastruktur_ogd.fahrleitungen_fahrdraht',
 'vbz_infrastruktur_ogd.strecke_steuerungskabine',
 'vbz_infrastruktur_ogd.strecke_esaerdanlage',
 'vbz_infrastruktur_ogd.fahrleitungen_tragwerk_punkt',
 'vbz_infrastruktur_ogd.strecke_tr_kreuzung',
 'vbz_infrastruktur_ogd.fahrleitungen_mauerbolzen',


In [28]:
vbz_layers = []
for layer in fiona.listlayers(data_dir + '/raw_data/geodata_stadt_Zuerich/vbz/data/data.gpkg'):
    gdf = gpd.read_file(data_dir + '/raw_data/geodata_stadt_Zuerich/vbz/data/data.gpkg', layer=layer)
    # Check if the first geometry is a LineString
    if gdf.geometry.iloc[0].geom_type == 'LineString':
        vbz_layers.append(gdf)

# Merge all the layers
vbz = pd.concat(vbz_layers)

/home/silas/miniconda3/envs/geo/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [27]:
vbz_layers

[   art     arttext                                 name inbetriebsetzung  \
 0    1  Tramtunnel  Tramtunnel Schwammendingen einwärts             None   
 1    1  Tramtunnel  Tramtunnel Schwammendingen auswärts             None   
 
    objectid                                           geometry  
 0   57356.0  LINESTRING (2683332.070 1250362.790, 2683334.5...  
 1   57357.0  LINESTRING (2683415.575 1250436.262, 2683415.5...  ,
        quellid einbaudatum  aktualitaet lagequalitaet  orientierung  objectid  \
 0        16505  2006-01-01  nachgeführt       ungenau          90.0     16699   
 1        16506  2006-01-01  nachgeführt       ungenau          90.0     16700   
 2        16507  2006-01-01  nachgeführt       ungenau          90.0     16701   
 3        16508  2006-01-01  nachgeführt       ungenau          90.0     16702   
 4        16509  2006-01-01  nachgeführt       ungenau          90.0     16703   
 ...        ...         ...          ...           ...           ...       .

In [25]:
vbz

,art,arttext,name,inbetriebsetzung,objectid,geometry,quellid,einbaudatum,aktualitaet,lagequalitaet,...,kraftende,besonderheit,besonderheittext,haltebereich,haltebereichtext,ueberholbarkeit,ueberholbarkeittext,bus,tram,mvu
0,1.0,Tramtunnel,Tramtunnel Schwammendingen einwärts,None,57356.0,"LINESTRING (2683332.070 1250362.790, 2683334.5...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Tramtunnel,Tramtunnel Schwammendingen auswärts,None,57357.0,"LINESTRING (2683415.575 1250436.262, 2683415.5...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,16699.0,POINT (2684882.637 1248178.029),16505.0,2006-01-01,nachgeführt,ungenau,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,16700.0,POINT (2684882.439 1248177.039),16506.0,2006-01-01,nachgeführt,ungenau,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,16701.0,POINT (2684887.554 1248180.095),16507.0,2006-01-01,nachgeführt,ungenau,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174075,NaN,NaN,NaN,NaN,47328.0,"LINESTRING (2682578.927 1246644.164, 2682578.2...",NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174076,NaN,NaN,NaN,NaN,47329.0,"LINESTRING (2682574.216 1246637.022, 2682573.7...",NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174077,NaN,NaN,NaN,NaN,47330.0,"LINESTRING (2682570.326 1246630.206, 2682568.4...",NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174078,NaN,NaN,NaN,NaN,47331.0,"LINESTRING (2682651.926 1246748.733, 2682584.8...",NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import MultiPolygon
from rasterstats import zonal_stats

def suitability_analysis(
    buffer_dist_vbz: float,
    buffer_trees: float,
    max_slope: float,
    parking_lots_path: str,
    slope_path: str,
    tree_locations_path: str,
    vbz_line_paths: list[str],
    vbz_point_paths: list[str],
    output_path: str
):
    # Read input data
    parking_lots = gpd.read_file(parking_lots_path)
    trees = gpd.read_file(tree_locations_path)
    
    # Step 1: Filter parking types
    parking_filtered = parking_lots[
        ~parking_lots['parking'].isin(['underground', 'multi-storey'])
    ].copy()

    # Step 2: Process VBZ features
    # Merge and buffer line features
    vbz_lines = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_line_paths]),
        crs=parking_lots.crs
    )
    vbz_line_buffers = vbz_lines.buffer(buffer_dist_vbz)

    # Merge and buffer point features
    vbz_points = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_point_paths]),
        crs=parking_lots.crs
    )
    vbz_point_buffers = vbz_points.buffer(buffer_dist_vbz)

    # Step 3: Buffer trees
    tree_buffers = trees.buffer(buffer_trees)

    # Step 4: Combine all buffers
    all_buffers = gpd.GeoSeries(
        list(vbz_line_buffers) + list(vbz_point_buffers) + list(tree_buffers),
        crs=parking_lots.crs
    ).unary_union

    # Step 5: Zonal statistics for slope
    with rasterio.open(slope_path) as src:
        slope_stats = zonal_stats(
            parking_filtered,
            slope_path,
            stats=['mean', 'max'],
            nodata=src.nodata
        )

    # Add slope stats to parking lots
    parking_filtered['slope_mean'] = [s['mean'] for s in slope_stats]
    parking_filtered['slope_max'] = [s['max'] for s in slope_stats]

    # Step 6: Filter by slope
    slope_filtered = parking_filtered[parking_filtered['slope_mean'] <= max_slope]

    # Step 7: Spatial difference with buffers
    final_areas = slope_filtered.geometry.difference(all_buffers)

    # Step 8: Calculate areas and filter
    result = gpd.GeoDataFrame(geometry=final_areas, crs=parking_lots.crs)
    result['area'] = result.geometry.area
    suitable = result[result['area'] >= 16]

    # Save output
    suitable.to_file(output_path, driver='GeoJSON')
    return suitable

# Example usage:
suitability_analysis(
    buffer_dist_vbz=2,
    buffer_trees=5,  # Example tree buffer distance
    max_slope=10,    # Example max slope
    parking_lots_path='parking.gpkg',
    slope_path='slope.tif',
    tree_locations_path='trees.gpkg',
    vbz_line_paths=['vbz_lines1.gpkg', 'vbz_lines2.gpkg'],
    vbz_point_paths=['vbz_points.gpkg'],
    output_path='suitable_parking.geojson'
)


ModuleNotFoundError: No module named 'numpy.distutils'

In [14]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from shapely.geometry import mapping

def suitability_analysis(
    buffer_dist_vbz: float,
    buffer_trees: float,
    max_slope: float,
    parking_lots_path: str,
    slope_path: str,
    tree_locations_path: str,
    vbz_line_paths: list[str],
    vbz_point_paths: list[str],
    output_path: str
):
    # Read input data
    parking_lots = gpd.read_file(parking_lots_path)
    trees = gpd.read_file(tree_locations_path)

    # Step 1: Filter parking types
    parking_filtered = parking_lots[
        ~parking_lots['parking'].isin(['underground', 'multi-storey'])
    ].copy()

    # Step 2: Process VBZ features
    # Merge and buffer line features
    vbz_lines = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_line_paths]),
        crs=parking_lots.crs
    )
    vbz_line_buffers = vbz_lines.buffer(buffer_dist_vbz)

    # Merge and buffer point features
    vbz_points = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_point_paths]),
        crs=parking_lots.crs
    )
    vbz_point_buffers = vbz_points.buffer(buffer_dist_vbz)

    # Step 3: Buffer trees
    tree_buffers = trees.buffer(buffer_trees)

    # Step 4: Combine all buffers
    all_buffers = gpd.GeoSeries(
        list(vbz_line_buffers) + list(vbz_point_buffers) + list(tree_buffers),
        crs=parking_lots.crs
    ).unary_union

    # Step 5: Custom zonal statistics for slope
    with rasterio.open(slope_path) as src:
        # Ensure CRS match
        parking_filtered = parking_filtered.to_crs(src.crs)
        
        slope_means = []
        slope_maxs = []
        
        for _, row in parking_filtered.iterrows():
            geom = [mapping(row.geometry)]
            
            try:
                # Extract raster values within polygon
                out_image, out_transform = mask(
                    src, geom, crop=True, nodata=src.nodata
                )
                values = out_image[0].flatten()
                valid_values = values[values != src.nodata]
                
                if valid_values.size > 0:
                    slope_means.append(np.nanmean(valid_values))
                    slope_maxs.append(np.nanmax(valid_values))
                else:
                    slope_means.append(np.nan)
                    slope_maxs.append(np.nan)
            except ValueError:
                slope_means.append(np.nan)
                slope_maxs.append(np.nan)

    # Add slope stats to parking lots
    parking_filtered['slope_mean'] = slope_means
    parking_filtered['slope_max'] = slope_maxs

    # Step 6: Filter by slope
    slope_filtered = parking_filtered[parking_filtered['slope_mean'] <= max_slope]

    # Step 7: Spatial difference with buffers
    final_areas = slope_filtered.geometry.difference(all_buffers)

    # Step 8: Calculate areas and filter
    result = gpd.GeoDataFrame(geometry=final_areas, crs=parking_lots.crs)
    result['area'] = result.geometry.area
    suitable = result[result['area'] >= 16]

    # Save output
    suitable.to_file(output_path, driver='GeoJSON')
    return suitable
